In [67]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 18 11:58:07 2017

@author: Biagio Brattoli
"""
import numpy as np
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image


img_max = 96
tile = 25
puzzle_full = 75

class DataLoader(data.Dataset):
    def __init__(self, data_path, txt_list, classes=1000):
        self.data_path = data_path
        self.names, _ = self.__dataset_info(txt_list)
        self.N = len(self.names)
        self.permutations = self.__retrive_permutations(classes)

        self.__image_transformer = transforms.Compose([
            transforms.Resize(img_max, Image.BILINEAR),
            transforms.CenterCrop(img_max - 1)])
        self.__augment_tile = transforms.Compose([
            transforms.RandomCrop(tile - 4),
            transforms.Resize((25, 25), Image.BILINEAR),
            transforms.Lambda(rgb_jittering),
            transforms.ToTensor(),
            # transforms.Normalize(mean=[0.485, 0.456, 0.406],
            # std =[0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]

        img = Image.open(framename).convert('RGB')
        
        if np.random.rand() < 0.30:
            img = img.convert('LA').convert('RGB')

        if img.size[0] != img_max:
            img = self.__image_transformer(img)

        s = float(img.size[0]) / 3
        print(s)
        a = s / 2
        tiles = [None] * 9
        
        for n in range(9):
            i = n / 3
            j = n % 3
            c = [a * i * 2 + a, a * j * 2 + a]
            c = np.array([c[1] - a, c[0] - a, c[1] + a + 1, c[0] + a + 1]).astype(int)
            tile = img.crop(c.tolist())
            tile = self.__augment_tile(tile)
            # Normalize the patches indipendently to avoid low level features shortcut
            m, s = tile.view(3, -1).mean(dim=1).numpy(), tile.view(3, -1).std(dim=1).numpy()
            s[s == 0] = 1
            norm = transforms.Normalize(mean=m.tolist(), std=s.tolist())
            tile = norm(tile)
            tiles[n] = tile

        order = np.random.randint(len(self.permutations))
        data = [tiles[self.permutations[order][t]] for t in range(9)]
        data = torch.stack(data, 0)

        return data, int(order), tiles

    def __len__(self):
        return len(self.names)

    def __dataset_info(self, txt_labels):
        with open(txt_labels, 'r') as f:
            images_list = f.readlines()
        
        file_names = []
        labels = []
        for row in images_list:
            row = row.split('\n')
            file_names.append(row[0])
            labels.append(int(1))

        return file_names, labels

    def __retrive_permutations(self, classes):
        all_perm = np.load('permutations_%d.npy' % (classes))
        # from range [1,9] to [0,8]
        if all_perm.min() == 1:
            all_perm = all_perm - 1

        return all_perm


def rgb_jittering(im):
    im = np.array(im, 'int32')
    for ch in range(3):
        im[:, :, ch] += np.random.randint(-2, 2)
    im[im > img_max] = img_max
    im[im < 0] = 0
    return im.astype('uint8')


In [68]:
ad = "../sslime/extra_scripts/data"

trainpath = ad +'/train'
train_data = DataLoader(trainpath, ad +'/train.txt',
                            classes=1000)

In [69]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                            batch_size= 1,
                                            shuffle=True,
                                            num_workers=1)

In [70]:
len(train_loader)

5000

In [71]:
for i, (images, labels, original) in enumerate(train_loader):
    print("i = ", i)
    print(images.shape)
    print(labels.shape)
    print(len(original))
    break

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x2adbca0c82e8>>
Traceback (most recent call last):
  File "/home/lhm300/.conda/envs/slime_env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/lhm300/.conda/envs/slime_env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/lhm300/.conda/envs/slime_env/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


32.0
32.0
i =  0
torch.Size([1, 9, 3, 25, 25])
torch.Size([1])
9


In [72]:
print(images[0][0][:].shape)

torch.Size([3, 25, 25])


In [73]:
import matplotlib.pyplot as plt

In [74]:
print(np.reshape(images[0][0][:], (25,25,3)).shape)
img = np.reshape(images[0][0][:], (3,25,25))

torch.Size([25, 25, 3])
